## 문자-단위 RNN 으로 이름 생성

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

In [2]:
all_letters = string.ascii_letters + ".,;'-"
n_letters = len(all_letters) + 1 # EOS  기호를 추가하여 갯수를 센다

In [5]:
def findFiles(path): return glob.glob(path)

def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
  if unicodedata.category(c) != 'Mn'
  and c in all_letters
  )

#  파일을 읽어 줄 단위로 분리
def readLines(filename):
  lines = open(filename, encoding = 'utf-8').read().strip().split('\n')
  return [unicodeToAscii(line) for line in lines]

# 각 언어의 이름 목록인 category_lines 사전 생성
category_lines = {}
all_categories = []
for filename in findFiles('/content/drive/My Drive/korean_embedding/raw_data/data/data/names/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Vietnamese', 'Dutch', 'Portuguese', 'Greek', 'Irish', 'French', 'Japanese', 'Scottish', 'Italian', 'Russian', 'Chinese', 'Spanish', 'English', 'Arabic', 'Polish', 'Korean', 'German', 'Czech']
O'Neal


## 네트워크 생성
- RNN 이 다른 입력들과 연결되는 category tensor 를 추가 인자로 가질 수 있도록 확장
- category tensor 는 문자 입력과 마찬가지로 one-hot 벡터
- 출력물이 다음 문자의 예측 확률로 사용

In [6]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        # 카테고리 , 인풋, 히든 정보를 모두 합침
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)

        # hidden & output 을 합쳐 하나의 output 으로 추가 생성
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1) # 0.1 수준으로 드롭아웃
        self.softmax = nn.LogSoftmax(dim=1) # 활성화 함수 소프트맥스 씌워주는 역할

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

## 학습
- (category - line) 무작위 쌍을 얻음
- 각 시점마다 (학습 단어의 문자 마다) 
  - input :  (언어, 현재 문자, 은닉 상태 ) category, input, hidden
  - output : (다음 문자, 다음 은닉 상태)  output, hidden
- 각 시점마다 현재 문자에서 다음 문자를 예측하기 때문에, 문자 쌍은 한 줄(하나의 이름) 에서 연속된 문자 그룹
  - 예: ABCE<EOS> 의 경우, (A,B), (B, C), (C, E), (E, <EOS>) 로 생성
- category 텐서는 1 x n_categories 크기의 원 핫 텐서로 생성됨
  - 모든 시간 단계 (시점) 에서 네트워크에 전달

In [7]:
import random

# 목록에서 무작위 아이템 반환
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# 임의의 category 및 그 category에서 무작위 줄(이름) 얻기
def randomTrainingPair():
    category = randomChoice(all_categories) # 랜덤한 category 명이 등장
    line = randomChoice(category_lines[category]) # 해당 카테고리에 대한 line 도 랜덤하게 선택됨
    return category, line

In [9]:
# Category를 위한 One-hot 벡터
def categoryTensor(category):
    
    # all_categories 는 리스트 형태이기 때문에 .index 로 인덱스를 불러와
    # 그 기준으로 원-핫 텐서를 만들 수 있다
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# 입력을 위한 처음부터 마지막 문자(EOS 제외)까지의  One-hot 행렬
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    
    # 각 글자에 대한 원-핫 텐서 만들어줌
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# 목표를 위한 두번째 문자 부터 마지막(EOS) 까지의 LongTensor
def targetTensor(line):

    # 그 다음 글자를 꺼내줌 : 0이 아니라 1부터 꺼냄 
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    
    # 제일 마지막엔 따로 <EOS> 를 위해 <EOS> 의 자리를 반환
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)


In [10]:
# 임의의 Category에서 Category, Input, Target Tensor를 만듭니다.
def randomTrainingExample():
    
    # 임의로 카테고리와 그 카테고리 중 한 단어를 뽑아옴
    category, line = randomTrainingPair()

    # 간단하게 카테고리 텐서를 만들어줌
    category_tensor = categoryTensor(category)

    # 인풋, 아웃풋 텐서를 별도로 만들어준 뒤 반환
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

## 네트워크 학습
- 모든 단계에서 예측을 수행하므로 모든 단계에서 손실을 계산
  - 마지막 출력만 사용하는 분류가 아니므로!
- augograd 를 사용하면, 각 단계의 손실을 간단하게 합쳐 역전파 진행도 가능함!!!!!

In [11]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)


In [12]:
import time # 얼마나 걸리는지 확인하기 위함
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)